<span type="title">指令系统体系和结构</span> | <span type="update">2018-07-12</span>

<span type="intro"><p type="card-text">本章介绍x86和MIPS体系的指令结构和汇编语言实现。对于x86，讲解了16、32、64位的不同结构以及其结构内的实现。对于MIPS，介绍了R、I、J、运算、访存、控制9种不同的指令类型，并对常用指令进行了详细介绍。</p></span>

# 设计自己的计算机

## 定义指令

一个计算机的软硬件接口必须能够提供三种操作：进行运算、数据传送、条件转移。这三种分别对应运算类指令、传送类指令和转移类指令。

- ADD R,M 将M内容和R内容相加，保存在R中
- LOAD R,M 将M的内容放入R中
- STORE M,R 将R的内容存回M中
- JMP L 无条件转向L处

（M,L为存储器地址，R为寄存器编号）

## 指令实现

每条指令占有2个字节，也就是16位，第一个字节前四位为操作码，代表LOAD、STORE等操作，后四个字节为寄存器编号，第二个字节共8位为存储单元的地址。根据这个定义，操作码可以有16条，寄存器编号最多16个，存储单元地址最多2^8 = 256字节。

比如：`LOAD R3, [5] ==> 0000 0011 00000101`其中是分别对应的。

![](w2p1.png)

如上是在模型机中的指令实现，其中每条指令包含两个字节，如存储器所示。软硬件协商在开机的时候PC读取对应位置的代码，开始执行，一般为00000000。

# x86体系结构

x86的体系可以分为三阶段，从1978年起的x86-16/IA-16，16位宽地址的CPU结构；从1885年起的IA-32,地址宽度扩充为32位，；从2003年开始的x86-64/AMD-64，CPU宽度扩充为64位。

## 8086体系结构

**概述**

8086是Intel较为成功的x86商业处理器，其内部通用寄存器位宽16位，可以处理8位或者16位数据，物理地址采用“段+偏移”的方式实现（位宽不足以完成全部内存寻址）。其数据总线16bit，地址总线20bit。

8086的寄存器模型由通用多功能寄存器（对应R）、指令指针寄存器（对应PC）、标志寄存器（对应F）和段寄存器构成。

![](w2p4.png)

**数据寄存器AX,AH,AL**

其数据寄存器有四个，均为16位，每个16位寄存器可分为2个8位使用，编号以AX,BX,CX,DX构成，其中AX可分为AH和AL两个8位寄存器，下同。其中A-D代表含义不同，A代表Accumulator，存储乘除指令的操作数，Base存档存储单元偏移地址，Count存放计数值，Data存放惩罚的部分积和除法的被除数。

**标志寄存器FLAGS**

标志寄存器包含若干标志位，其中分为两类，其一为状态标志，表示CPU工作状态，比如运算是否进位，结果是否为0，控制标志是对CPU运行起特定控制作用的标志，比如连续还是单步，是否中断。

8086的标志位如下：

![](w2p2.png)

其中比较重要的是CF进位标志和ZF零标志。

**指令指针寄存器IP**

Instruction Pointer，保存一个内存地址，指向当前需要取出的指令。程序员不能对IP直接存取，转移指令、返回指令等会改变IP内容。其寻址能力为2^16=64K。而20位地址线的寻址能力为2^20=1M字节。

**段寄存器CS**

Segment Register，用来和其他寄存器联合生成存储器地址。其中，

CS 称为代码段寄存器 Code Segment； DS 称为 Data Segment 数据段寄存器，ES 称为 Extra Segment 附加段寄存器， SS 称为 Stack Segment 堆栈段寄存器。

**地址生成方式**

8086的地址生成方式为段基值进位后加上偏移量，其中段基值保存在段寄存器中，偏移量保存在程序地址中，这样就可以通过地址总线存取真实物理地址了（完成从逻辑地址到物理地址映射）。

比如：保存在DS中的[2000H]加上IR中的 MOV AX, [3000H], 得到 [23000H],然后通过地址总线传输到MAR中获取对应值。

![](w2p3.png)

## 80386 体系结构

80386支持了32位的算术和逻辑运算，提供32位通用存储器。地址总线为32位，支持4GB内存空间。

其扩展了原来的 AH,AL->AX->EAX 等通用存储器，也扩展了 IP -> EIP, FLAG -> EFLAG 这两个指针和标志寄存器，同时添加了FS和GS两个段寄存器，遗憾的是，为了兼容之前的架构，一直使用这种逻辑地址通过段寄存器获取实际地址的获取地址方式。

![](w2p5.png)

## x86-64 体系结构

![](w2p6.png)

64位则在EAX基础上添加了RAX，IP和FLAGS也添加了RIP和RFLAGS，通用寄存器还增加了R8-R15。

# x86 指令实现

指令共分为四类，包括尽心算术和逻辑运算的运算类指令、从存储器到寄存器的传送类指令、暂停处理器、清除标志位的控制类指令、用于IF语句等的转移类指令这四种。

指令的效果有：改变F（产生标志位），改变R，改变M，改变指令指针等。

以一段程序来举例：

![](w2p7.png)

这段程序将存储器两个数相加，放在另外一个地址中。红色表示第一个int，蓝色表示第二个int，绿色标识int这种类型/数字长度。

## 传送类指令

这类指令用于将数据或地址放到存储器或者寄存器，如下：

![](w2p8.png)

比如， `MOV DST, SRC` 将 SRC 的值传送给 DST。

x86的汇编灵活性很强，这里的DST和SRC，可以直接为寄存器from操作数、可以为寄存器地址from寄存器地址、可以为寄存器from存储器地址、也可以为存储器地址指向的寄存器名称from寄存器地址，更可以为一段简单的代码from寄存器地址。

![](w2p9.png)

MOV指令不是定长的，如下所示，代表了汇编语言和其二进制对应代码的含义。

![](w2p10.png)

如下三句话：

```
MOV CL, [2500H]
MOV SI, 2000H
MOV DI, 3000H
```

CL代表用于Count的L通用寄存器，大小为8位，存储数组长度。
分别将2000H地址的数据和3000H地址的数据传送到SI和DI这两个通用寄存器。

## 运算类指令

如下所示：

![](w2p11.png)

![](w2p12.png)

ADD 指令表示将两个数相加，不考虑进位，而ADC 指令则考虑进位，其用到上一次CF中的位值，加上操作数和目标数，得到结果。INC 指令表示将地址中寄存器数值加1。

对于如下三句话：

```
CLC
LOOP1:
    MOV AX, [SI]
    ADC AX, [DI]
    MOV [SI], AX
    INC SI
    INC SI
    INC DI
    INC DI
    DEC CL
```
进行了进位标志位CF的清零，然后将第一个数放到AX这个32位宽的寄存器中，将第二个数和AX相加，并将这个结果保存到[SI]寄存器中。之后使用INC对于这两个数的寄存器进行+1（相当于循环的a++，这里进行两次INC因为一个数占据了两个字节），然后对保存数长度的CL寄存器-1（相当于循环的i）。

## 转义类指令

其中分为条件转义和无条件转移，直接转移和间接转移，其目的都是为了改变指令执行的顺序。

![](w2p13.png)

![](w2p14.png)

对应我们的程序：

```
JNZ LOOP1 循环执行累加操作
```

JNZ 会在遇到 ZF 零标志时停止执行。

## 控制类指令

![](w2p15.png)

## 复杂的x86指令举例

串操作指令是将存储器中的数据串进行每一个元素的操作。串的基本单位是字节，其长度可达64kb。串可以加前缀，组合复杂的功能。

常见的串操作指令有：

![](w2p16.png)

`REP MOVSB ` 

REP: 当CX≠0的时候，重复MOVSB

MOVSB: 表示将存储器特定位置的一个字节单元传送到另一个位置。其内含的操作数为：DS:SI -> ES:DI，串的长度在CX寄存器中。硬件自动修改SI和DI的值，指向下一个串元素，使用重复前缀时，CX自动递减。

![](w2p17.png)

其中，CLD 和 STD 的含义如下：

![](w2p18.png)

其中 DF 存储在 F 中。

方向标志的作用在于，如果目的串和源串地址重合，那么指定方向可以避免转移错误。

# MIPS 架构简介

Microprocessor without Interlocked Piped Stages

没有互锁的流水线微处理器

其主要特点有：

- 固定的指令长度（32-bit，即1 word）
    - 简化了从存储器取指令
- 简单的寻址模式
    - 简化了从存储器取操作数
- 指令数量少，指令功能简单（一条指令只完成一个操作）
    - 简化指令的执行过程
- 只有Load和Store指令可以访问存储器
    - 例如，不支持x86指令的这种操作：ADD AX,[3000H]
- 需要优秀的编译器支持


## MIPS指令基本格式

对于MIPS,其所有指令如下，根据对象分为R, I, J 三种，而每一种则分别可以进行运算、访存和分支三种指令，如下表所示：

![](w2p21.png)

## 运算指令

`add a,b,c` 将b和c的结果求和，放入a中。

![](w2p19.png)

## 访存指令

```
◦ A是一个100个字（word）的数组，首地址在寄存器$19中
◦ 变量h对应寄存器$18
◦ 临时数据存放在寄存器$8

A[10]=h+A[3] 对应的MIPS指令为：
- lw $8,12($19) # t0=A[3]
- add $8,$18,$8 # t0=h+A[3]
- sw $8,40($19) # A[10]=h+A[3]
```

如果横着看的话，根据控制对象的不同，MIPS由由R、I、J三种组成，如下所示，其中Register表示寄存器，Immediate表示立即数，Jump表示无条件转移。

![](w2p20.png)

## R型指令

如上上图所示，其包含6个域，

其中前6bit存放opcode，而后6bit存放funct，opcode帮助确认指令类型，对于R指令，其opcode均为0，funct用于精确指定指令类型，和opcode配合使用。

5bit rs Source Register 用于指定第一个寄存器编号

5bit rt Target Register 用于指定第二个寄存器编号

5bit rd 表示目的操作数寄存器编号（保存运算结果）

其中5bit的域可以表示0-31，对应32个通用寄存器。

shamt shift amount 用于指定移位指令的位数，没有移位，则为0。

对于add来说，其含义如下：

![](w2p22.png)

## I型指令

I型指令大部分和R类似，不过，其立即数占据16bit，包含了原来在R中的rd、shamt和funct。

对于访存指令，如lw rt,imm(rs)
通常可以满足访存地址偏移量的需求（-32768~+32767）

对于运算指令，如addi rt,rs,imm
无法满足全部需求，但大多数时候可以满足需求

对于addi来说，其含义如下：

![](w2p23.png)


## 分支指令

条件分支用于改变控制流，基于结果，比如 branch if equal（beq）,branch if not equal(bne)。

非条件分支用于无条件改变控制流，比如 jump(j)

对于 beq 来说：

![](w2p24.png)

```mips
//c语言
if(i==j)
    f=g+h;
else
    f=g-h;

//mips语言 

beq $s3,$s4,True # branch i==j
sub $s0,$s1,$s2 # f=g-h(false)
j Fin # goto Fin
True: add $s0,$s1,$s2 # f=g+h (true)
```

对于非条件分支（R型）而言，其最远表示2^28 = 256MB地址，为了到达更远的地址，使用jr指令或者2次调用j指令。